# Neu3D : FFBH2019 - Project
`Neu3D` ([NPM](https://www.npmjs.com/package/neu3d), [GitHub](https://github.com/fruitflybrain/neu3d)) is a 3D SWC visualization package. In it's current incarnation, Neu3D focuses on Fruitfly brain visualization and provides a collection of user interactions that drives `NeuroNLP`. 

You can fork the [repo](https://github.com/fruitflybrain/neu3d) to get a copy of the Neu3D package. 

## Installation

Do the following steps to 
```
git clone your-neu3d-fork
cd neu3d
npm install 
npm run build
python -m http.server 8000
```
The last step launches an http server on port 8000, which can be accessed by visiting `localhost:8000/examples/larva.html` in your browser.

**Note**: We recommend **Chrome** for development of Neu3D.
**Note**: The swc files provided are for Drosophila larva and have a different center and scale as opposed to the adult data (`examples/adult.html`).

## Development
Use the following tips for development:
1. Run `npm run watch` in a terminal different from the one you're launching http server from. This allows for continuously update and re-compile Neu3D.
2. Make use of Chrome's DevTools for de-bugging and testing purposes.
3. Make sure to disable cache when developing. See [Here](https://stackoverflow.com/questions/5690269/disabling-chrome-cache-for-website-development)

### Example - Add function to animate bloompass strength
We demonstrate how to add functionalities to the neu3d library with a simply example:
1. Edit `neu3d/index.js` by adding the `prototype` functions to the `Neu3D` class
2. Test the custom function by executing `neu3d.animateBloomPass` in the console

You should see the objects fluctuating in bloom pass strength when you execute the function.


```javascript
// neu3d/index.js

import { Neu3D } from './src/neu3d';                                                            

// Beginning your code here

/**                                                                                             
 * Custom Function to animate Bloom Pass                                                   
 *                                                                                              
 * @param interval time inveral (ms) at which the bloompass is updated                          
 * @param period   period (ms) of bloom pass oscillation                                        
 * @param amplitude bloompass value is in range [0,amplitude]                                   
 */                                                                                             
Neu3D._animateBPTimer = undefined;                                                              
Neu3D.prototype.animateBloomPass = function (interval=100, period=5000, amplitude=1){           
    if (this._animateBPTimer === undefined){                                                    
        this._animateBPTimer = setInterval(()=>{                                                
            let d = new Date();                                                                 
            let bp = amplitude * (1+Math.sin( 2*3.1415*(d.getTime() % period)/period));         
            this.settings.bloomPass.strength = bp;                                              
        }, interval);                                                                           
    }                                                                                           
}                                                                                               
                                                                                                
Neu3D.prototype.stopBloomPass = function(clear=false){                                          
    clearInterval(this._animateBPTimer);                                                        
                                                                                                
    if (clear === true){ // reset to default value                                              
        this.settings.bloomPass.strength = 0.2;                                                 
    }                                                                                           
    this._animateBPTimer = undefined;                                                           
}                    

// end of your code

export { Neu3D };
```

You can also extend the control panel by defining a function that adds a [font-awesome](https://fontawesome.com/) button (`fa fa-sun` in this case) with a callback to toggle Bloom Pass Animation.

```javascript
// neu3d/index.js

import { Neu3D } from './src/neu3d';                                                            

// Beginning your code here
/**                                                                                             
 * Custom Function to animate Bloom Pass                                             
 *                                                                                              
 * @param interval time inveral (ms) at which the bloompass is updated                          
 * @param period   period (ms) of bloom pass oscillation                                        
 * @param amplitude bloompass value is in range [0,amplitude]                                   
 */                                                                                             
Neu3D._animateBPTimer = undefined;                                                              
Neu3D.prototype.animateBloomPass = function (interval=100, period=1000, amplitude=1){           
    if (this._animateBPTimer === undefined){                                                    
        this._animateBPTimer = setInterval(()=>{                                                
            let d = new Date();                                                                 
            let bp = amplitude * (1+Math.sin( 2*3.1415*(d.getTime() % period)/period));         
            this.settings.bloomPass.strength = bp;                                              
        }, interval);                                                                           
    }                                                                                           
}                                                                                               
                                                                                                
Neu3D.prototype.stopBloomPass = function(clear=false){                                          
    clearInterval(this._animateBPTimer);                                                        
                                                                                                
    if (clear === true){ // reset to default value                                              
        this.settings.bloomPass.strength = 0.2;                                                 
    }                                                                                           
    this._animateBPTimer = undefined;                                                           
}                                                                                               
                                                                                                
                                                                                                
/**                                                                                             
 * Add a button to toggle animateBloomPass                                                      
 */                                                                                             
Neu3D.prototype.addBloomPassBtn= function(){                                                    
    const _createBtn = (name, icon, iconAttrs, tooltip, func) =>{                               
        let newButton = function () {                                                           
            this[name] = func;                                                                  
        };                                                                                      
        let btn = new newButton();                                                              
        let buttonid = this.controlPanel.add(btn, name).title(tooltip).icon(icon,"strip",iconAt\
trs);                                                                                           
        return buttonid;                                                                        
    }                                                                                           
                                                                                                
    let newBtn = _createBtn("toggleBP", "far fa-sun", {}, "Toggle Animate BloomPass",           
                            () => {                                                             
                                                                                                
                                if (this._animateBPTimer === undefined){                        
                                    this.animateBloomPass();                                    
                                }else{                                                          
                                    this.stopBloomPass();                                       
                                }                                                               
                            }                                                                   
                           );                                                                   
    window.newBtn = newBtn;                                                                     
    // this is a hacky way to add the UI button.                                                
    // Neu3D.controlPanel.__ul gives an unordered list of all buttons in the dat.GUI.           
    // children[2] points to the first UI button.                                               
    this.controlPanel.__ul.insertBefore(newBtn.__li, this.controlPanel.__ul.children[2]);       
}


// end of your code
export { Neu3D };
```

Now if you execute `neu3d.addBloomPassBtn()` in console, you will be greeted with the new panel. Notice that the first button now toggle Bloom Pass Animation.

Alternatively, you can add `neu3d.addBloomPassBtn()` to your `index.html` file and have it automatically loaded:
```html
// neu3d/index.html
...
var neu3d = new Neu3D(parentDiv,                                                          
                       undefined,                                                       
                       { "globalCenter": { 'x': 0, 'y': -250, 'z': 0 } },               
                       options);                                                        
neu3d.addBloomPassBtn();
...
```

<img src='figures/settings_2.png' width=500px>

## Development Ideas

A few ideas to get you started:
1. Animation:
    * Add additional animation functions, e.g. orbit camera around object, etc.
    * See [Here](https://github.com/mkturkcan/Cameraman) for an example. Note that the example assumes the existance of `ffbomesh` object, which is the same as the `neu3d` instance  
2. Visualization:
    * Add background 
    * Create rendering presets 

## Reference: [SWC](http://research.mssm.edu/cnic/swc.html) Format
SWC format files are formatted as:
```
1 1 14.566132 34.873772 7.857000 0.717830 -1
2 0 16.022520 33.760513 7.047000 0.463378 1
3 5 17.542000 32.604973 6.885001 0.638007 2
4 0 19.163984 32.022469 5.913000 0.602284 3
5 0 20.448090 30.822802 4.860000 0.436025 4
6 6 21.897903 28.881084 3.402000 0.471886 5
7 0 18.461960 30.289471 8.586000 0.447463 3
8 6 19.420759 28.730757 9.558000 0.496217 7
...
```
where the columns are  
> **n T x y z R P**

and they refer to :
* **n**: an integer label that identifies the current point and increments by one from one line to the next.
* **T**: an integer representing the type of neuronal segment, such as soma, axon, apical dendrite, etc. The standard accepted integer values are given below.
    - 0 = undefined
    - 1 = soma
    - 2 = axon
    - 3 = dendrite
    - 4 = apical dendrite
    - 5 = fork point
    - 6 = end point
    - 7 = custom
* **x, y, z**: the cartesian coordinates of each node.
* **R**: the radius at that node.
* **P**: the parent (the integer label) of the current point or -1 to indicate an origin (soma).

## Reference: Neu3D Usage
After building `Neu3D` and launching http server, navigate to `localhost:8000` and you'll see the following figure

<img src='figures/neu3d.png' width=500px>

### User Interactions
A few operators are exposed to the user view the modified [dat.GUI](https://github.com/dataarts/dat.gui) module shown on the top right of the page.

<img src='figures/settings.png' width=400px>

From _left to right_, the buttons are:
1. Upload SWC file
2. Reset View to default canvas
3. Reset View to visible objects in the workspace
4. Hide all objects
5. Show all objects
6. Take Screenshot
7. Remove all unpinned neurons
8. Unpin all neurons
9. Toggle settings menu

### Settings
The visualization settings directly affect the rendering of objects in the canvas. The main option here is to toggle 3D rendering of neurons. 
To improve rendering efficiency, neurons are by default rendered as 2D wires since 3D objects require significantly more CPU/GPU resources for rendering. 


<img src='figures/neu3d_settings.png' style='height:500px'>